# Hello Tensorflow 2.0

### Installing
```bash
pip install tensorflow-datasets
pip install tensorflow-gpu==2.0.0-alpha0
```

### Starting Tensorboard
```bash
tensorboard --logdir=logs/gradient_tape
```

### References
* https://medium.com/tensorflow/introducing-tensorflow-datasets-c7f01f7e19f3
* https://www.youtube.com/watch?v=P4_rJfHpr7k&list=PLQY2H8rRoyvzoUYI26kHmKSJBedn3SQuB
* https://medium.com/tensorflow/upgrading-your-code-to-tensorflow-2-0-f72c3a4d83b5
* https://www.tensorflow.org/tutorials/eager/automatic_differentiation
* https://www.tensorflow.org/tutorials/eager/eager_basics
* https://www.tensorflow.org/tensorboard/r2/get_started

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
import datetime

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

num_epochs = 5

# Delete previous logs
!rm -rf ./logs/ 

### Configure Tensorboard

In [2]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = 'logs/gradient_tape/' + current_time + '/train'
test_log_dir = 'logs/gradient_tape/' + current_time + '/test'
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)

### Load Data from Tensorflow public datasets Repositories

In [3]:
dataset, info = tfds.load('mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = dataset['train'], dataset['test']

### Pre-process the data

In [4]:
def convert_types(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

# Configure mini-batch of 32 elements and apply function to convert the data
mnist_train = mnist_train.map(convert_types).shuffle(10000).batch(32)
mnist_test = mnist_test.map(convert_types).batch(32)

### Define Model

In [5]:
class MyModel(Model):
    # Like Pytorch instantiate here the layers
    def __init__(self, num_classes):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(num_classes, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

In [6]:
# Intiantiate model
model = MyModel(10)

# Define loss and optimizer
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

### Define Metrics to debug training

In [7]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

### Define Training and Validation steps

In [8]:
@tf.function
def train_step(image, label):
    # Record operations for automatic-differentiation
    with tf.GradientTape() as tape:
        # Apply Model and get predictions
        predictions = model(image)
        # Calculate Loss
        loss = loss_object(label, predictions)
        
    # Calculate the gradient of the loss with respect to the model parameters
    gradients = tape.gradient(loss, model.trainable_variables)
    # Run optimizer
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # Run Debug Metrics
    train_loss(loss)
    train_accuracy(label, predictions)

@tf.function
def test_step(image, label):
    # Apply Model and get predictions
    predictions = model(image)
    t_loss = loss_object(label, predictions)

    test_loss(t_loss)
    test_accuracy(label, predictions)

### Train loop

In [9]:
# For each epoch
for epoch in range(num_epochs):
    # Train
    for image, label in mnist_train:
        train_step(image, label)
        # Send some stuff to tensorboard
        with train_summary_writer.as_default():
            tf.summary.scalar('loss', train_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)


    # Validate
    for test_image, test_label in mnist_test:
        test_step(test_image, test_label)
        with test_summary_writer.as_default():
            tf.summary.scalar('loss', test_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)

    # Print some information
    template = 'Epoch {}:\n\tLoss: {}, Acc: {}\n\tTest Loss: {}, Test Acc: {}'
    print (template.format(epoch+1,
                         train_loss.result(), 
                         train_accuracy.result()*100,
                         test_loss.result(), 
                         test_accuracy.result()*100))

Epoch 1:
	Loss: 0.14727464318275452, Acc: 95.59833526611328
	Test Loss: 0.06305709481239319, Test Acc: 98.08999633789062
Epoch 2:
	Loss: 0.09768898785114288, Acc: 97.06999969482422
	Test Loss: 0.05951105058193207, Test Acc: 98.17500305175781
Epoch 3:
	Loss: 0.07308969646692276, Acc: 97.78666687011719
	Test Loss: 0.055514197796583176, Test Acc: 98.27999877929688
Epoch 4:
	Loss: 0.05875635892152786, Acc: 98.21500396728516
	Test Loss: 0.05596238374710083, Test Acc: 98.29750061035156
Epoch 5:
	Loss: 0.04887521639466286, Acc: 98.51399993896484
	Test Loss: 0.0561077781021595, Test Acc: 98.3479995727539


### Get Summary of the Model and Save

In [10]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  320       
_________________________________________________________________
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  2769024   
_________________________________________________________________
dense_1 (Dense)              multiple                  1290      
Total params: 2,770,634
Trainable params: 2,770,634
Non-trainable params: 0
_________________________________________________________________
